# Assignment #3: A simple language classifier

Author: Pierre Nugues

## Objectives

In this assignment, you will implement a language detector inspired from Google's _Compact language detector_, version 3 (CLD3): https://github.com/google/cld3. CLD3 is written in C++ and its code is available from GitHub. The objectives of the assignment are to:
* Write a program to classify languages
* Use neural networks
* Know what a classifier is
* Write a short report of 1 to 2 pages to describe your program. You will notably comment the performance you obtained and how you could improve it.

## Description

### System Overview

Read the GitHub description of CLD3, https://github.com/google/cld3, (_Model_ section). In your individual report you will:
1. Summarize the system in two or three sentences;
2. Outline the CLD3 overall architecture in a figure. Use building blocks only and do not specify the parameters.

## Imports

In [2]:
import bz2
import json
import os
import numpy as np
import requests
from sklearn.neural_network import MLPClassifier
from sklearn.feature_extraction import DictVectorizer
from sklearn.metrics import accuracy_score
from sklearn.metrics import f1_score, classification_report
from sklearn.metrics import confusion_matrix

### Dataset

As dataset, we will use Tatoeba, https://tatoeba.org/eng/downloads. It consists of more than 8 million short texts in 347 languages and it is available in one file called `sentences.csv`.

The dataset is structured this way: There is one text per line, where each line consists of the three following fields separated by tabulations and ended by a carriage return:
```
sentence id [tab] language code [tab] text [cr]
```
Each text (sentence) has a unique id and has a language code that follows the ISO 639-3 standard (see below). 

### Scope of the lab

In this lab, you will consider three languages only: French (fra), English (eng), and Swedish (swe). Below is an excerpt of the Tatoeba dataset limited to these three languages: 

```
1276    eng     Let's try something.
1277    eng     I have to go to sleep.
1280    eng     Today is June 18th and it is Muiriel's birthday!
...
1115    fra     Lorsqu'il a demandé qui avait cassé la fenêtre, tous les garçons ont pris un air innocent.
1279    fra     Je ne supporte pas ce type.
1441    fra     Pour une fois dans ma vie je fais un bon geste... Et ça ne sert à rien.
...
337413  swe     Vi trodde att det var ett flygande tefat.
341910  swe     Detta är huset jag bodde i när jag var barn.
341938  swe     Vi hade roligt på stranden igår.
...
```
Tatoeba is updated continuously. The examples from this dataset come from a corpus your instructor downloaded on September 15, 2020.

### Understanding the $\mathbf{X}$ matrix (feature matrix)

You will now investigate the CLD3 features:
 *  What are the features CLD3 extracts from each text?
 * Create manually a simplified $\mathbf{X}$ matrix where you will represent the 9 texts with CLD3 features. You will use a restricted set of features: You will only consider the letters _a_, _b_, and _n_ and the bigrams _an_, _ba_, and _na_. You will ignore the the rest of letters and bigrams as well as the trigrams. Your matrix will have 9 rows and 6 columns, each column will contain these counts: `[#a, #b, #n, #an, #ba, #na]`.

The CLD3's original description uses relative frequencies (counts of a letter divided by the total counts of letters in the text). Here, you will use the raw counts. To help you, your instructor filled the fourth row of the matrix corresponding to the first text in French. Fill in the rest. You will include this matrix in your report. 

$\mathbf{X} =
\begin{bmatrix}
-& -& -& -&-& -\\
-& -& -& -&-& -\\
-& -& -& -&-& -\\
8& 0& 8& 1&0&0\\
-& -& -& -&-& -\\
-& -& -& -&-& -\\
-& -& -& -&-& -\\
-& -& -& -&-& -\\
-& -& -& -&-& -\\
\end{bmatrix}$
; $\mathbf{y} =
\begin{bmatrix}
     \text{eng} \\
     \text{eng}\\
     \text{eng}\\
    \text{fra}\\
   \text{fra}  \\
     \text{fra}\\
    \text{swe}\\
 \text{swe}   \\
 \text{swe}   
\end{bmatrix}$

## Programming: Extracting the features

Before you start programming, download the Tatoeba dataset.

### Loading and filtering the dataset

Run the code to read the dataset and split it into lines. You may have to change the path

In [16]:
dataset = open('sentences.tar/sentences.csv', encoding='utf8').read().strip()
dataset = dataset.split('\n')
dataset[:10]

['1\tcmn\t我們試試看！',
 '2\tcmn\t我该去睡觉了。',
 '3\tcmn\t你在干什麼啊？',
 '4\tcmn\t這是什麼啊？',
 '5\tcmn\t今天是６月１８号，也是Muiriel的生日！',
 '6\tcmn\t生日快乐，Muiriel！',
 '7\tcmn\tMuiriel现在20岁了。',
 '8\tcmn\t密码是"Muiriel"。',
 '9\tcmn\t我很快就會回來。',
 '10\tcmn\t我不知道。']

Run the code to split the fields and remove possible whitespaces

In [17]:
dataset = list(map(lambda x: tuple(x.split('\t')), dataset))
dataset = list(map(lambda x: tuple(map(str.strip, x)), dataset))
dataset[:3]

[('1', 'cmn', '我們試試看！'), ('2', 'cmn', '我该去睡觉了。'), ('3', 'cmn', '你在干什麼啊？')]

Write the code to extract the French, English, and Swedish texts. You will call the resulting dataset: `dataset_small`

In [21]:
# Write your code here
dataset_small = []
for tp in dataset:
    if tp[1] == 'swe' or tp[1] == 'eng' or tp[1] == 'fra':
        dataset_small.append(tp)

NameError: name 'dataset' is not defined

In [22]:
del dataset
dataset_small[0:5]

NameError: name 'dataset' is not defined

### Functions to Count Characters Ngrams

Write a function `count_chars(string, lc=True)` to count characters (unigrams) of a string. You will set the text in lowercase if `lc` is set to `True`. As in CLD3, you will return the relative frequencies of the unigrams.

In [77]:
# Write your code here
def count_chars(string, lc=True):
    if lc == True:
        string = string.lower()

    char_dict = {}
    tot_nbr_char = len(string)
    for i in range(tot_nbr_char):
        if string[i] in char_dict:
            char_dict[string[i]] = char_dict[string[i]] + 1
        else: 
            char_dict[string[i]] = 1

    for key in char_dict:
        char_dict[key] = char_dict[key]/tot_nbr_char

    return char_dict

Write a function `count_bigrams(string, lc=True)` to count the characters bigrams of a string. You will set the text in lowercase if `lc` is set to `True`. As in CLD3, you will return the relative frequencies of the bigrams.

In [18]:
# Write your code here
def count_bigrams(string, lc=True):
    if lc == True:
        string = string.lower()

    char_dict = {}
    tot_nbr_bigrams = len(string)-1
    for i in range(tot_nbr_bigrams):
        bigram = string[i] + string[i+1]
        if bigram in char_dict:
            char_dict[bigram] = char_dict[bigram] + 1
        else: 
            char_dict[bigram] = 1

    for key in char_dict:
        char_dict[key] = round(char_dict[key]/tot_nbr_bigrams, 5)

    return char_dict

Write a function `count_trigrams(string, lc=True)` to count the characters trigrams of a string. You will set the text in lowercase if `lc` is set to `True`. As in CLD3, you will return the relative frequencies of the trigrams.

In [19]:
# Write your code here
def count_trigrams(string, lc=True):
    if lc == True:
        string = string.lower()

    char_dict = {}
    tot_nbr_trigrams = len(string) - 2
    for i in range(tot_nbr_trigrams):
        trigram = string[i] + string[i+1] + string[i+2]
        if trigram in char_dict:
            char_dict[trigram] = char_dict[trigram] + 1
        else: 
            char_dict[trigram] = 1

    for key in char_dict:
        char_dict[key] = round(char_dict[key]/tot_nbr_trigrams, 5)

    return char_dict

In [78]:
count_chars("Let's try something.")

{'l': 0.05,
 'e': 0.1,
 't': 0.15,
 "'": 0.05,
 's': 0.1,
 ' ': 0.1,
 'r': 0.05,
 'y': 0.05,
 'o': 0.05,
 'm': 0.05,
 'h': 0.05,
 'i': 0.05,
 'n': 0.05,
 'g': 0.05,
 '.': 0.05}

In [27]:
count_bigrams("Let's try something.")

{'le': 0.05263,
 'et': 0.10526,
 "t'": 0.05263,
 "'s": 0.05263,
 's ': 0.05263,
 ' t': 0.05263,
 'tr': 0.05263,
 'ry': 0.05263,
 'y ': 0.05263,
 ' s': 0.05263,
 'so': 0.05263,
 'om': 0.05263,
 'me': 0.05263,
 'th': 0.05263,
 'hi': 0.05263,
 'in': 0.05263,
 'ng': 0.05263,
 'g.': 0.05263}

In [35]:
count_trigrams("Let's try something.")

{'let': 0.05556,
 "et'": 0.05556,
 "t's": 0.05556,
 "'s ": 0.05556,
 's t': 0.05556,
 ' tr': 0.05556,
 'try': 0.05556,
 'ry ': 0.05556,
 'y s': 0.05556,
 ' so': 0.05556,
 'som': 0.05556,
 'ome': 0.05556,
 'met': 0.05556,
 'eth': 0.05556,
 'thi': 0.05556,
 'hin': 0.05556,
 'ing': 0.05556,
 'ng.': 0.05556}

### Counting the ngrams in the dataset

You will now extract the features from each text. For this, add the character, bigram, and trigram relative frequencies to the texts using this format:
`(text_id, language_id, text, char_cnt, bigram_cnt, trigram_cnt)`.

From the datapoint:
`('1276', 'eng', "Let's try something.")`,
you must return:

`('1276', 'eng', "Let's try something.", 
  {'l': 0.05, 'e': 0.1, 't': 0.15, "'": 0.05, 's': 0.1, ' ': 0.1, 'r': 0.05, 'y': 0.05, 'o': 0.05, 'm': 0.05, 'h': 0.05, 'i': 0.05, 'n': 0.05, 'g': 0.05, '.': 0.05},
  {'le': 0.05263157894736842, 'et': 0.10526315789473684, "t'": 0.05263157894736842, "'s": 0.05263157894736842, 's ': 0.05263157894736842, ' t': 0.05263157894736842, 'tr': 0.05263157894736842, 'ry': 0.05263157894736842, 'y ': 0.05263157894736842, ' s': 0.05263157894736842, 'so': 0.05263157894736842, 'om': 0.05263157894736842, 'me': 0.05263157894736842, 'th': 0.05263157894736842, 'hi': 0.05263157894736842, 'in': 0.05263157894736842, 'ng': 0.05263157894736842, 'g.': 0.05263157894736842},
  {'let': 0.05555555555555555, "et'": 0.05555555555555555, "t's": 0.05555555555555555, "'s ": 0.05555555555555555, 's t': 0.05555555555555555, ' tr': 0.05555555555555555, 'try': 0.05555555555555555, 'ry ': 0.05555555555555555, 'y s': 0.05555555555555555, ' so': 0.05555555555555555, 'som': 0.05555555555555555, 'ome': 0.05555555555555555, 'met': 0.05555555555555555, 'eth': 0.05555555555555555, 'thi': 0.05555555555555555, 'hin': 0.05555555555555555, 'ing': 0.05555555555555555, 'ng.': 0.05555555555555555})`

You will store the extracted features in a list that you will call `dataset_small_feat`

In [79]:
# Write your code here
dataset_small_feat = []
for tp in dataset_small:
    dataset_small_feat.append((tp[0], tp[1], tp[2], count_chars(tp[2])))

In [80]:
del dataset_small
dataset_small_feat[:2]

[('1115',
  'fra',
  "Lorsqu'il a demandé qui avait cassé la fenêtre, tous les garçons ont pris un air innocent.",
  {'l': 0.044444444444444446,
   'o': 0.05555555555555555,
   'r': 0.05555555555555555,
   's': 0.07777777777777778,
   'q': 0.022222222222222223,
   'u': 0.044444444444444446,
   "'": 0.011111111111111112,
   'i': 0.06666666666666667,
   ' ': 0.16666666666666666,
   'a': 0.08888888888888889,
   'd': 0.022222222222222223,
   'e': 0.05555555555555555,
   'm': 0.011111111111111112,
   'n': 0.08888888888888889,
   'é': 0.022222222222222223,
   'v': 0.011111111111111112,
   't': 0.05555555555555555,
   'c': 0.022222222222222223,
   'f': 0.011111111111111112,
   'ê': 0.011111111111111112,
   ',': 0.011111111111111112,
   'g': 0.011111111111111112,
   'ç': 0.011111111111111112,
   'p': 0.011111111111111112,
   '.': 0.011111111111111112}),
 ('1276',
  'eng',
  "Let's try something.",
  {'l': 0.05,
   'e': 0.1,
   't': 0.15,
   "'": 0.05,
   's': 0.1,
   ' ': 0.1,
   'r': 0.05,
  

The unigram frequencies

In [81]:
dataset_small_feat[0][3].items()

dict_items([('l', 0.044444444444444446), ('o', 0.05555555555555555), ('r', 0.05555555555555555), ('s', 0.07777777777777778), ('q', 0.022222222222222223), ('u', 0.044444444444444446), ("'", 0.011111111111111112), ('i', 0.06666666666666667), (' ', 0.16666666666666666), ('a', 0.08888888888888889), ('d', 0.022222222222222223), ('e', 0.05555555555555555), ('m', 0.011111111111111112), ('n', 0.08888888888888889), ('é', 0.022222222222222223), ('v', 0.011111111111111112), ('t', 0.05555555555555555), ('c', 0.022222222222222223), ('f', 0.011111111111111112), ('ê', 0.011111111111111112), (',', 0.011111111111111112), ('g', 0.011111111111111112), ('ç', 0.011111111111111112), ('p', 0.011111111111111112), ('.', 0.011111111111111112)])

The bigram frequencies

In [16]:
# dataset_small_feat[0][4].items()

dict_items([('L', 0.011), ('o', 0.056), ('r', 0.056), ('s', 0.078), ('q', 0.022), ('u', 0.044), ("'", 0.011), ('i', 0.067), ('l', 0.033), (' ', 0.167), ('a', 0.089), ('d', 0.022), ('e', 0.056), ('m', 0.011), ('n', 0.089), ('é', 0.022), ('v', 0.011), ('t', 0.056), ('c', 0.022), ('f', 0.011), ('ê', 0.011), (',', 0.011), ('g', 0.011), ('ç', 0.011), ('p', 0.011), ('.', 0.011)])

## Programming: Building $\mathbf{X}$

You will now build the $\mathbf{X}$ matrix. In this assignment, you will only consider unigrams to speed up the training step. This means that you will set aside the character bigrams and trigrams.

When you are done with the lab requirements, feel free to improve the program and include bigrams and trigrams. To add bigrams, a possible method is to add the bigram dictionary to the unigram one using update and then to extract the resulting dictionary. You can easily extend this to trigrams. Feel free to use another method if you want.

In [ ]:
INCLUDE_BIGRAMS = False
if INCLUDE_BIGRAMS:
    for i in range(len(dataset_small_feat)):
        dataset_small_feat[i][3].update(dataset_small_feat[i][4])

### Vectorizing the features

The CLD3 architecture uses embeddings. In this lab, we will simplify it and we will use a feature vector instead consisting of the character frequencies. For example, you will represent the text:

`"Let's try something."`

with:

`{'l': 0.05, 'e': 0.1, 't': 0.15, "'": 0.05, 's': 0.1, ' ': 0.1, 
 'r': 0.05, 'y': 0.05, 'o': 0.05, 'm': 0.05, 'h': 0.05, 'i': 0.05, 
 'n': 0.05, 'g': 0.05, '.': 0.05}`

To create the $\mathbf{X}$ matrix, we need to transform the dictionaries of `dataset_small` into numerical vectors. The `DictVectorizer` class from the scikit-learn library, see here [https://scikit-learn.org/stable/modules/generated/sklearn.feature_extraction.DictVectorizer.html], has two methods, `fit()` and `transform()`, and a combination of both `fit_transform()` to convert dictionaries into such vectors.

You will now write the code to:

1. Extract the character frequency dictionaries from `dataset_small` corresponding to its 3rd index and set them in a list;
2. Convert the list of dictionaries into an $\mathbf{X}$ matrix using `DictVectorizer`.

#### Extracting the character frequencies

Produce a new list of datapoints with the unigrams only. Each item in this list will be a dictionary. You will call it `X_cat`

In [82]:
# Write your code here
X_cat  = []
for tp in dataset_small_feat:
    X_cat.append(tp[3])

In [83]:
X_cat[:2]

[{'l': 0.044444444444444446,
  'o': 0.05555555555555555,
  'r': 0.05555555555555555,
  's': 0.07777777777777778,
  'q': 0.022222222222222223,
  'u': 0.044444444444444446,
  "'": 0.011111111111111112,
  'i': 0.06666666666666667,
  ' ': 0.16666666666666666,
  'a': 0.08888888888888889,
  'd': 0.022222222222222223,
  'e': 0.05555555555555555,
  'm': 0.011111111111111112,
  'n': 0.08888888888888889,
  'é': 0.022222222222222223,
  'v': 0.011111111111111112,
  't': 0.05555555555555555,
  'c': 0.022222222222222223,
  'f': 0.011111111111111112,
  'ê': 0.011111111111111112,
  ',': 0.011111111111111112,
  'g': 0.011111111111111112,
  'ç': 0.011111111111111112,
  'p': 0.011111111111111112,
  '.': 0.011111111111111112},
 {'l': 0.05,
  'e': 0.1,
  't': 0.15,
  "'": 0.05,
  's': 0.1,
  ' ': 0.1,
  'r': 0.05,
  'y': 0.05,
  'o': 0.05,
  'm': 0.05,
  'h': 0.05,
  'i': 0.05,
  'n': 0.05,
  'g': 0.05,
  '.': 0.05}]

#### Vectorize `X_cat`

Convert you `X_cat` matrix into a numerical representation using `DictVectorizer`: https://scikit-learn.org/stable/modules/generated/sklearn.feature_extraction.DictVectorizer.html

In [84]:
# Write your code here
v = DictVectorizer(sparse=True)
X = v.fit_transform(X_cat)

## Programming: Building $\mathbf{y}$

You will now convert the list of language symbols into a $\mathbf{y}$ vector

Extract the language symbols from `dataset_small_feat` and call the resulting list `y_cat`

In [85]:
# Write your code here
y_cat = []
for tp in dataset_small_feat:
    y_cat.append(tp[1])

In [86]:
y_cat[:5]

['fra', 'eng', 'eng', 'fra', 'eng']

Extract the set of language symbols and build two indices mapping the symbols to integers and the integers to symbols. Both indices will be dictionaries that you will call: `lang2inx`and `inx2lang`.

In [87]:
# Write your code here
inx2lang = {0: 'fra', 1: 'eng', 2: 'swe'}
lang2inx = {'fra': 0, 'eng': 1, 'swe': 2}

In [88]:
inx2lang

{0: 'fra', 1: 'eng', 2: 'swe'}

In [89]:
lang2inx

{'fra': 0, 'eng': 1, 'swe': 2}

Convert your `y_cat` vector into a numerical vector. Call this vector `y`.

In [90]:
# Write your code here
y = []
for lang in y_cat:
    y.append(lang2inx[lang])

In [92]:
y[:5]

[0, 1, 1, 0, 1]

## Programming: Building the Model

Create a neural network using sklearn with a hidden layer of 50 nodes and a relu activation layer: https://scikit-learn.org/stable/modules/neural_networks_supervised.html. Set the maximal number of iterations to 5, in the beginning, and verbose to True. Use the default values for the rest. You will call you classifier `clf`

In [93]:
# Write your code here
clf = MLPClassifier(hidden_layer_sizes=(50,), max_iter=5, verbose=True)

In [94]:
clf

MLPClassifier(hidden_layer_sizes=(50,), max_iter=5, verbose=True)

### Training and Validation Sets

You will now split the dataset into training and a validation sets

#### We shuffle the indices

In [95]:
indices = list(range(X.shape[0]))
np.random.shuffle(indices)
print(indices[:10])
X = X[indices, :]
y = np.array(y)[indices]

[227873, 727261, 139438, 1499258, 307620, 774643, 206487, 379810, 1556788, 111656]


#### We split the dataset

In [96]:
training_examples = int(X.shape[0] * 0.8)

X_train = X[:training_examples, :]
y_train = y[:training_examples]

X_val = X[training_examples:, :]
y_val = y[training_examples:]

### Fitting the model

Fit the model on the training set

In [115]:
# Write your code here
clf.fit(X_train, y_train)

Iteration 1, loss = 0.13613843
Iteration 2, loss = 0.07482120
Iteration 3, loss = 0.07042567
Iteration 4, loss = 0.06670668
Iteration 5, loss = 0.06401141


C:\Users\johan\anaconda3\lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:582: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (5) reached and the optimization hasn't converged yet.
  warnings.warn(


MLPClassifier(hidden_layer_sizes=(50,), max_iter=5, verbose=True)

## Predicting

Predict the `X_val` languages. You will call the result `y_val_pred`

In [116]:
# Write your code here
y_val_pred = clf.predict(X_val)

In [117]:
print(y_val_pred[:10])
print(y_val[:10])

[1 1 1 1 1 1 1 1 1 1]
[1 1 1 1 1 1 1 1 1 0]


#### Evaluating

In [118]:
# evaluate the model
accuracy_score(y_val, y_val_pred)

0.9771813690088489

In [119]:
y_symbols = ['fra', 'eng', 'swe']
print(classification_report(y_val, y_val_pred, target_names=y_symbols))
print('Micro F1:', f1_score(y_val, y_val_pred, average='micro'))
print('Macro F1', f1_score(y_val, y_val_pred, average='macro'))

              precision    recall  f1-score   support

         fra       0.96      0.95      0.96     87853
         eng       0.98      0.99      0.98    271594
         swe       0.96      0.90      0.93      7490

    accuracy                           0.98    366937
   macro avg       0.97      0.95      0.96    366937
weighted avg       0.98      0.98      0.98    366937

Micro F1: 0.9771813690088489
Macro F1 0.957442829254163


### Confusion Matrix

In [120]:
confusion_matrix(y_val, y_val_pred)

array([[ 83871,   3930,     52],
       [  3432, 267952,    210],
       [    81,    668,   6741]], dtype=int64)

Increase the number of iterations to improve the score. You may also change the parameters.

## Predict the language of a text

Now predict the languages of the strings below.

In [121]:
docs = ["Salut les gars !", "Hejsan grabbar!", "Hello guys!", "Hejsan tjejer!"]

Create features vectors from this list. Call this matrix `X_test`

In [122]:
# Write your code here
hello = []
for sentence in docs:
    hello.append(count_chars(sentence))
    
X_test = v.transform(hello)

And run the prediction that you will store in a variable called `pred_languages`

In [123]:
# Write your code here
pred_languages = clf.predict(X_test) # print(X_test.shape, X_val.shape, X_train.shape) för att kolla dimentionerna
y = []
for lang_code in pred_languages:
    y.append(inx2lang[lang_code])
pred_languages = y

In [124]:
pred_languages

['fra', 'swe', 'eng', 'swe']

## Submission

When you have written all the code and run all the cells, fill in your ID and as well as the name of the notebook.

In [126]:
STIL_ID = ["elt15jli", "elt15afa"] # Write your stil ids as a list
CURRENT_NOTEBOOK_PATH = os.path.join(os.getcwd(), 
                                     "3-language_detector.ipynb") # Write the name of your notebook

The submission code will send your answer. It consists of the predicted languages.

In [127]:
ANSWER = json.dumps({'pred_langs': pred_languages})
ANSWER

'{"pred_langs": ["fra", "swe", "eng", "swe"]}'

Now the moment of truth:
1. Save your notebook and
2. Run the cells below

In [128]:
SUBMISSION_NOTEBOOK_PATH = CURRENT_NOTEBOOK_PATH + ".submission.bz2"

In [129]:
ASSIGNMENT = 3
API_KEY = "f581ba347babfea0b8f2c74a3a6776a7"

# Copy and compress current notebook
with bz2.open(SUBMISSION_NOTEBOOK_PATH, mode="wb") as fout:
    with open(CURRENT_NOTEBOOK_PATH, "rb") as fin:
        fout.write(fin.read())

In [130]:
res = requests.post("https://vilde.cs.lth.se/edan20checker/submit", 
                    files={"notebook_file": open(SUBMISSION_NOTEBOOK_PATH, "rb")}, 
                    data={
                        "stil_id": STIL_ID,
                        "assignment": ASSIGNMENT,
                        "answer": ANSWER,
                        "api_key": API_KEY,
                    },
               verify=True)

# from IPython.display import display, JSON
res.json()

{'msg': None,
 'status': 'correct',
 'signature': 'f20542a48d07ce3a67a0bf035aa5121e21e6caf42b9daec4c67ce3348b9f20e41bd7374f4536d06fc0803952e2ac74ebd0ec90b2ac7f6f4434b5fccd72cdc20e',
 'submission_id': '7a8f1cdd-1509-49f3-93f6-b100ba15476f'}

## Postscript from Pierre Nugues

I created this assignment from an examination I wrote last year for the course on applied machine learning. I simplified it from the `README.md` on GitHub, https://github.com/google/cld3. I found the C++ code difficult to understand and I reimplemented a Keras/Tensorflow version of it from this `README`. Should you be interested, you can find it here: https://github.com/pnugues/language-detector.